In [1]:
import pandas as pd
import tensorflow_data_validation as tfdv

In [2]:
from __future__ import print_function

In [3]:
dataset = pd.read_csv('../input/pollution-small.csv')

In [4]:
dataset.shape

(2188, 5)

In [5]:
training_data = dataset[:1600]

In [6]:
training_data.describe()

,pm10,no2,so2,soot
count,1600.000000,1600.000000,1600.000000,1600.000000
mean,49.656494,30.980519,16.229981,21.551956
std,35.211906,12.400788,10.621896,12.127354
min,6.380000,9.740000,4.010000,6.000000
25%,28.345000,22.567500,9.777500,14.400000
50%,38.835000,28.715000,13.275000,18.630000
75%,58.050000,36.370000,19.282500,24.072500
max,277.250000,138.010000,123.130000,107.650000


In [7]:
test_data = dataset[1600:]

In [8]:
test_data.describe()

,pm10,no2,so2,soot
count,588.000000,588.000000,588.000000,588.000000
mean,44.648248,37.296922,13.605170,18.441310
std,28.992087,10.940050,5.098944,6.596459
min,11.900000,15.070000,4.990000,8.000000
25%,28.337500,29.217500,10.122500,14.410000
50%,35.555000,35.815000,12.345000,17.090000
75%,50.812500,43.872500,15.855000,20.962500
max,273.770000,106.030000,38.030000,87.210000


## Paso 1: Analisis descriptivo Tensorflow Data Validation (tfdv)

In [9]:
# Esquema del set de datos, todos los estadisticos descriptivos se almacenan aqui
train_stats = tfdv.generate_statistics_from_dataframe(dataframe = training_data)

#### Inferir el esquema

In [10]:
schema = tfdv.infer_schema(statistics = train_stats)

In [11]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Date',BYTES,required,,-
'pm10',FLOAT,required,,-
'no2',FLOAT,required,,-
'so2',FLOAT,required,,-
'soot',FLOAT,required,,-


### Calcular los estadísticos descriptivos del conjunto de testing

In [12]:
test_stats = tfdv.generate_statistics_from_dataframe(dataframe = test_data)

## Paso 2: Comparar estadísticos del conjunto de test con Schema

### Buscar anomalías en los nuevos datos

In [13]:
# Se estarán evaluando anomalias en función al esquema de entrenamiento,
# quiere decir que este indicador nos puede decir si hay discrepancias entre
# el Testing y el Training dataset
anomalies = tfdv.validate_statistics(statistics=test_stats, schema=schema)

### Mostrar todas las anomalias detectadas

- Número entero mayor que 10
- Tipo String cuando se espera un INT
- Tipo FLOAT cuando se espera un INT
- Número entero mayor que 0

In [14]:
tfdv.display_anomalies(anomalies)

### Nuevos datos con anomalias

In [17]:
# Creemos anomalias para evidenciar la naturaleza de display_anommalies
test_set_copy = test_data.copy()
# Ahora hacemos el daño
test_set_copy.drop("soot", axis=1, inplace=True)

### Estadísticos basados en datos con anomalías

In [19]:
 # Generar estadisticos de copia
test_set_copy_stats = tfdv.generate_statistics_from_dataframe(dataframe=test_set_copy)

# Validar anomalias
anomalies_copy = tfdv.validate_statistics(statistics=test_set_copy_stats, schema=schema) 

# Ver si hay anomalias
tfdv.display_anomalies(anomalies_copy) 

C:\Users\rafra\.conda\envs\tensorflow\lib\site-packages\tensorflow_data_validation\utils\display_util.py:173: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Anomaly short description,Anomaly long description
Feature name,,
'soot',Column dropped,Column is completely missing


## Paso 3: Preparar el esquema para subir a producción

In [20]:
# Creación de entornos para la fase entrendamiento y la recolección de datos
# de el servidor
schema.default_environment.append("TRAINING")
schema.default_environment.append("SERVING")

### Eliminar la columna objetivo del esquema de producción

In [21]:
# Del esquema conseguir la columna soot y
# la eliminamos del entorno de SERVING entonces
# no estará en el entorno del servidor
tfdv.get_feature(schema, 'soot').not_in_environment.append('SERVING')

### Comprobar anomalías entre el entorno del servidor y nuevos datos entrantes

In [23]:
# Utilizamos del esquema, la variante de entorno SERVING
serving_env_anomalies = tfdv.validate_statistics(test_set_copy_stats, schema, environment='SERVING')
tfdv.display_anomalies(serving_env_anomalies)

## Parte 4: Congelar el Esquema

In [26]:
tfdv.write_schema_text(schema = schema, output_path = 'pollution_schema.pbtxt')